In [ ]:
from flask import Flask, request, jsonify
from flask_restful import Resource, Api, reqparse
from flask_jwt import JWT
from flask_jwt import jwt_required

app = Flask(__name__)
app.secret_key = 'tym'
api = Api(app)

jwt = JWT(app, authenticate, identity) # new endpoint /auth

items = []

class Item(Resource):
    @jwt_required()
    def get(self, name):
        item = next(filter(lambda x: x['name'] == name, items), None) # first item found
        return {'item': item}, 200 if item else 404
        
    def post(self, name):
        if next(filter(lambda x: x['name'] == name, items), None) is not None:
            return {'message': "An item with name '{}' already exists".format(name)}, 400
        
        parser = reqparse.RequestParser()
        parser.add_argument('price', 
                            type=float, 
                            required=True, 
                            help='This field cannot be left blank'
        )
        data = parser.parse_args()
        new_item = {
                'name': name,
                'price': data['price']
            }
        items.append(new_item)
        return new_item, 201
    
    def delete(self, name):
        global items
        items = list(filter(lambda x: x['name'] != name, items)) # return list of items
        return {'message': "'{}' item deleted".format(name)}
    
    def put(self, name):
        parser = reqparse.RequestParser()
        parser.add_argument('price', 
                            type=float, 
                            required=True, 
                            help='This field cannot be left blank'
        )
        data = parser.parse_args()
        
        item = next(filter(lambda x: x['name'] == name, items), None)
        if item is None:
            new_item = {
                'name': name,
                'price': data['price']
            }
            items.append(new_item)
        else:
            item.update(data)
        
        return item, 201
    
class ItemList(Resource):
    def get(self):
        return {'items': items}

api.add_resource(Item, '/item/<string:name>') # http://127.0.0.1/itemt/<item name>
api.add_resource(ItemList, '/items')
app.run(port=5001)

 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)


In [ ]:
from werkzeug.security import safe_str_cmp
# user.py

class User(object):
    def __init__(self, id, username, password):
        self.id = id
        self.username = username
        self.password = password  

users = [
    User(1, 'bob', 'password')
]

username_mapping = {u.username: u for u in users}
userid_mapping = {u.id: u for u in users}

def authenticate(username, password):
    user = username_mapping.get(username, None)
    if user and safe_str_cmp(user.password, password):
        return user

def identity(payload):
    user_id = payload['identity']
    return userid_mapping.get(user_id, None)

